In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import time
import torchvision
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import os
import shutil
import random
from tqdm.notebook import tqdm
from pkg_resources import packaging

In [ ]:
num_epochs = 10
batch_size = 64
learning_rate = 0.001

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

In [ ]:
data_set = {
    'train':  datasets.ImageFolder("/kaggle/input/kaggle-cats-and-dogs1/kagglecatsanddogs_5340/PetImages", data_transforms['train']),
    'validation': datasets.ImageFolder("/kaggle/input/100-cats-and-dogs/Cats_and_Dogs", data_transforms['validation'])
}

In [ ]:
dataloaders = {
    'train':
    torch.utils.data.DataLoader(data_set['train'],
                                batch_size=128,
                                shuffle=True,
                                num_workers=0),
    'validation':
    torch.utils.data.DataLoader(data_set['validation'],
                                batch_size=8,
                                shuffle=False,
                                num_workers=0)
}

In [ ]:
model = torchvision.models.resnet50(weights=None)
model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
val_acc = []
train_acc = []
val_loss = []
train_loss = []
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            for inputs, labels in dataloaders[phase]:
                count += 1
                if(count % 100 == 0): print(count)
                inputs = inputs.cuda()
                labels = labels.cuda()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_set[phase])
            epoch_acc = running_corrects.double() / len(data_set[phase])
            if phase == 'train':
                train_acc.append(epoch_acc)
                train_loss.append(epoch_loss)
            if phase == 'validation':
                val_acc.append(epoch_acc)
                val_loss.append(epoch_loss)
            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

model_trained = train_model(model, criterion, optimizer, num_epochs=10)

In [ ]:
import matplotlib.pyplot as plt
epochs = [1,2,3,4,5,6,7,8,9,10]
plt.plot(epochs,val_acc)
plt.title("Epochs vs Accuracy")
plt.savefig("Epochs vs Accuracy.png")
plt.plot(epochs,val_loss)
plt.title("Epochs vs loss")
plt.savefig("Epochs vs loss.png")